In [5]:
import Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using AtomsIO
using DFTK
using PseudoPotentialData
using ForwardDiff
using DifferentiationInterface
using Optim

  Activating project at `~/git/tutorial-cecam-workshop-dftk-2025`


In [ ]:
system = load_system("GaAs_b3.extxyz")
pseudopotentials = PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf")
model0 = model_DFT(system; functionals=PBE(), pseudopotentials,
                   smearing=Smearing.Gaussian(), temperature=1e-3)

Atoms(AsGa, periodicity = TTT):
    cell_vectors      : [       0    2.875    2.875;
                            2.875        0    2.875;
                            2.875    2.875        0]u"Å"

    AtomView(Ga, [       0,        0,        0]u"Å")
    AtomView(As, [  1.4375,   1.4375,   1.4375]u"Å")


In [37]:
function strain_bandgap(η)
    model = Model(model0; lattice=(1 + η) * model0.lattice)
    basis = PlaneWaveBasis(model; Ecut=42, kgrid=(8, 8, 8))
    # basis = PlaneWaveBasis(model; Ecut=15, kgrid=(4,4,4))  # crude settings, for debugging
    scfres = self_consistent_field(basis; tol=1e-6)
    eigenvalues_Γ = scfres.eigenvalues[1]
    ε_vbm = maximum(eigenvalues_Γ[eigenvalues_Γ .≤ scfres.εF])
    ε_cbm = minimum(eigenvalues_Γ[eigenvalues_Γ .> scfres.εF])
    ε_cbm - ε_vbm
end

strain_bandgap (generic function with 1 method)

In [38]:
initial_bandgap = strain_bandgap(0.0)

┌ Warning: Negative ρcore detected: -3.2745383553565624e-8
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39


n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -182.5584474181                   -1.05    4.5    8.61s
  2   -182.5640231857       -2.25       -1.56    1.1    5.57s
  3   -182.5652835034       -2.90       -1.97    2.0    7.62s
  4   -182.5653148706       -4.50       -2.87    1.2    4.16s
  5   -182.5653188968       -5.40       -2.99    3.0    6.67s
  6   -182.5653191094       -6.67       -3.32    2.0    6.28s
  7   -182.5653191551       -7.34       -3.80    1.0    5.93s
  8   -182.5653191702       -7.82       -4.15    2.8    6.97s
  9   -182.5653191719       -8.76       -4.62    1.6    6.03s
 10   -182.5653191723       -9.42       -5.21    2.3    6.49s
 11   -182.5653191723      -10.65       -5.27    2.2    9.89s
 12   -182.5653191723      -11.09       -5.65    1.0    4.31s
 13   -182.5653191723      -11.46       -6.42    2.6    8.24s


0.005205828400278473

In [32]:
uconvert(u"eV", initial_bandgap * u"hartree")

4.217918335849347 eV

## Optimization of strain

In [36]:
η0 = [0.0]             # Initialize with zero strain (at equilibrium)
bandgap_target = 0.03  # Target band gap in Hartree
bandgap_loss(η) = (bandgap_target - strain_bandgap(η[1]))^2
res = Optim.optimize(bandgap_loss, η0, BFGS(),
                     Optim.Options(; iterations=5, x_abstol=1e-3, show_trace=true);
                     autodiff=AutoForwardDiff())
display(res)
println("Optimized design strain η = ", res.minimizer)

┌ Warning: Negative ρcore detected: Dual{ForwardDiff.Tag{typeof(bandgap_loss), Float64}}(-5.315189382870195e-7,-3.19635430124119e-5)
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39
┌ Warning: Negative ρcore detected: -5.315189382870195e-7
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39


n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -169.7940640133                    0.39    5.2    611ms
  2   -170.8973380805        0.04       -0.36    2.9    388ms
  3   -170.9131228602       -1.80       -1.24    2.6    341ms
  4   -170.9133576006       -3.63       -2.30    1.0    242ms
  5   -170.9133702027       -4.90       -3.39    2.1    265ms
  6   -170.9133720523       -5.73       -3.42    2.5    353ms
  7   -170.9133721497       -7.01       -3.80    1.0    231ms
  8   -170.9133721674       -7.75       -4.05    1.1    286ms
  9   -170.9133721753       -8.10       -4.44    1.6    963ms
 10   -170.9133721772       -8.72       -5.33    1.6    265ms
 11   -170.9133721774       -9.90       -5.30    2.2    330ms
 12   -170.9133721774      -10.93       -5.71    1.0    233ms
 13   -170.9133721774      -11.43       -6.21    1.6    259ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δt

┌ Warning: Negative ρcore detected: Dual{ForwardDiff.Tag{typeof(bandgap_loss), Float64}}(-8.468564563939656e-7,-4.824996886203849e-5)
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39
┌ Warning: Negative ρcore detected: -8.468564563939656e-7
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39


n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -169.6693289960                    0.39    5.1    1.10s
  2   -170.7874598487        0.05       -0.36    3.0    788ms
  3   -170.8025951067       -1.82       -1.26    2.5    673ms
  4   -170.8028207107       -3.65       -2.28    1.0    429ms
  5   -170.8028359900       -4.82       -3.16    2.2    796ms
  6   -170.8028370912       -5.96       -3.45    2.0    639ms
  7   -170.8028371751       -7.08       -3.82    1.0    266ms
  8   -170.8028371934       -7.74       -4.12    1.0    273ms
  9   -170.8028371991       -8.24       -4.45    1.7    299ms
 10   -170.8028372006       -8.83       -5.37    1.6    288ms
 11   -170.8028372007      -10.06       -5.46    2.2    418ms
 12   -170.8028372007      -11.06       -6.07    1.0    413ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ----

┌ Warning: Negative ρcore detected: Dual{ForwardDiff.Tag{typeof(bandgap_loss), Float64}}(-4.4385650081176625e-6,-0.00021983803487337898)
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39
┌ Warning: Negative ρcore detected: -4.4385650081176625e-6
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39


...

┌ Warning: Negative ρcore detected: Dual{ForwardDiff.Tag{typeof(bandgap_loss), Float64}}(-1.649156381071434e-6,-8.764628750269658e-5)
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39
┌ Warning: Negative ρcore detected: -1.649156381071434e-6
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39


n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -169.7760211075                    0.39    5.2    1.27s
  2   -170.8814443593        0.04       -0.37    2.9    475ms
  3   -170.8925183981       -1.96       -1.26    2.4    463ms
  4   -170.8928595875       -3.47       -2.26    1.3    318ms
  5   -170.8928735553       -4.85       -3.20    2.0    349ms
  6   -170.8928750711       -5.82       -3.24    2.5    448ms
  7   -170.8928752549       -6.74       -3.44    1.0    293ms
  8   -170.8928753800       -6.90       -3.74    1.0    282ms
  9   -170.8928754168       -7.43       -4.03    1.0    281ms
 10   -170.8928754291       -7.91       -4.69    1.6    316ms
 11   -170.8928754302       -8.94       -5.21    2.0    411ms
 12   -170.8928754302      -10.75       -5.73    1.1    297ms
 13   -170.8928754302      -11.33       -6.13    1.4    332ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δt

┌ Warning: Negative ρcore detected: Dual{ForwardDiff.Tag{typeof(bandgap_loss), Float64}}(-1.6595637723576824e-6,-8.813708424384778e-5)
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39
┌ Warning: Negative ρcore detected: -1.6595637723576824e-6
└ @ DFTK /home/schmitz/.julia/packages/DFTK/N6gvd/src/terms/xc.jl:39


n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -169.7664856913                    0.39    5.2    655ms
  2   -170.8730711832        0.04       -0.37    3.0    469ms
  3   -170.8841277844       -1.96       -1.25    2.4    408ms
  4   -170.8844759602       -3.46       -2.25    1.3    309ms
  5   -170.8844909043       -4.83       -3.19    1.9    342ms
  6   -170.8844925721       -5.78       -3.23    2.6    864ms
  7   -170.8844927510       -6.75       -3.44    1.0    582ms
  8   -170.8844928690       -6.93       -3.71    1.0    541ms
  9   -170.8844929086       -7.40       -3.99    1.0    487ms
 10   -170.8844929224       -7.86       -4.69    1.5    353ms
 11   -170.8844929236       -8.90       -5.24    2.1    509ms
 12   -170.8844929237      -10.68       -5.81    1.5    749ms
 13   -170.8844929237      -11.82       -6.22    1.6    330ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δt

 * Status: success

 * Candidate solution
    Final objective value:     1.497425e-02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 2.03e-04 ≤ 1.0e-03
    |x - x'|/|x'|          = 1.01e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.40e-07 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 9.35e-06 ≰ 0.0e+00
    |g(x)|                 = 5.55e-06 ≰ 1.0e-08

 * Work counters
    Seconds run:   488  (vs limit Inf)
    Iterations:    3
    f(x) calls:    12
    ∇f(x) calls:   12
